In [11]:
import usb.core
import usb.util
import struct
import numpy as np

# Find your device
dev = usb.core.find(idVendor=0x359F, idProduct=0x3031)

# Check if the device was found
if dev is None:
    dev = usb.core.find(idVendor=0x359F, idProduct=0x3021)
elif dev is None:
    dev = usb.core.find(idVendor=0x359F, idProduct=0x3031)
    if dev is None:
        raise ValueError('Device not found')

# Print the device speed
speed_dict = {
    1: "Low Speed",  # USB 1.0
    2: "Full Speed", # USB 1.1
    3: "High Speed", # USB 2.0
    4: "Super Speed" # USB 3.0
}

print(f'Device speed: {speed_dict.get(dev.speed, "Unknown Speed")}')

# Set the configuration
dev.set_configuration()

# Get an endpoint instance
cfg = dev.get_active_configuration()
intf = cfg[(0, 0)]

print(intf)

print('\n\n')

Voltage = 1.5

PWM_Val = int((Voltage / 3.3) * 1024)

value_bytes = struct.unpack("<2B", np.uint16(PWM_Val))
print(f'Set Voltage: {Voltage}V')
print(f'PWM 1to0 Value: {np.uint16(PWM_Val)} {[hex(x) for x in value_bytes]}')
print('\n\n')

bmRequestType = 0b01000000
bRequest = 1
wValue = 0x04
wIndex = 0
data = [0x00, 0x00, 0x00, 0x00]
data_hex = [hex(x) for x in data]
print('REG CTRL Write data:', data_hex)
# Perform control transfer
response = dev.ctrl_transfer(bmRequestType, bRequest, wValue, wIndex, data)
print('REG CTRL Write Bytes:', response)
print('\n')

bmRequestType = 0b01000000
bRequest = 1
wValue = 0x0C
wIndex = 0
data = [0x03, 0x00, 0x00, 0x00]
data_hex = [hex(x) for x in data]
print('REG AUX Write data:', data_hex)
# Perform control transfer
response = dev.ctrl_transfer(bmRequestType, bRequest, wValue, wIndex, data)
print('REG AUX Write Bytes::', response)

## VREF REG
# Define control transfer parameters
bmRequestType = 0b01000000
bRequest = 1
wValue = 0x10
wIndex = 0
data = list(value_bytes)
data_hex = [hex(x) for x in data]
print('VREF Write data:', data_hex)
# Perform control transfer
response1 = dev.ctrl_transfer(bmRequestType, bRequest, wValue, wIndex, data)

print('VREF Write Response:', response1)

# Release the device resources
usb.util.dispose_resources(dev)





Device speed: Super Speed
    INTERFACE 0: Vendor Specific ===========================
     bLength            :    0x9 (9 bytes)
     bDescriptorType    :    0x4 Interface
     bInterfaceNumber   :    0x0
     bAlternateSetting  :    0x0
     bNumEndpoints      :    0x2
     bInterfaceClass    :   0xff Vendor Specific
     bInterfaceSubClass :   0xff
     bInterfaceProtocol :   0xff
     iInterface         :    0x0 
      ENDPOINT 0x2: Bulk OUT ===============================
       bLength          :    0x7 (7 bytes)
       bDescriptorType  :    0x5 Endpoint
       bEndpointAddress :    0x2 OUT
       bmAttributes     :    0x2 Bulk
       wMaxPacketSize   :  0x400 (1024 bytes)
       bInterval        :    0x0
      ENDPOINT 0x82: Bulk IN ===============================
       bLength          :    0x7 (7 bytes)
       bDescriptorType  :    0x5 Endpoint
       bEndpointAddress :   0x82 IN
       bmAttributes     :    0x2 Bulk
       wMaxPacketSize   :  0x400 (1024 bytes)
       bInter